In [1]:
# Include libraries.
import paddle
import paddle.fluid as fluid
import numpy

In [2]:
SAVE_DIRNAME = 'models_'

In [3]:
# 输入输入层和标签层
x = fluid.layers.data(name='x', shape=[13], dtype='float32')

# 定义网络
y_predict = fluid.layers.fc(input=x, size=1, act=None)

y = fluid.layers.data(name='y', shape=[1], dtype='float32')

cost = fluid.layers.square_error_cost(input=y_predict, label=y)
avg_cost = fluid.layers.mean(cost)

# 定义优化器
sgd_optimizer = fluid.optimizer.SGD(learning_rate=0.001)
sgd_optimizer.minimize(avg_cost)


# 定义batch
train_reader = paddle.batch(
    paddle.reader.shuffle(paddle.dataset.uci_housing.train(), buf_size=500),
    batch_size=20)

# 定义场所，没有gpu的改成CPUPlace()
place = fluid.CUDAPlace(0)
exe = fluid.Executor(place)

# 定义双层循环
def train_loop():
    feeder = fluid.DataFeeder(place=place, feed_list=[x, y])
# 参数初始化
    exe.run(fluid.default_startup_program())

# PASS_NUM 运行多少轮数据
# 双层for循环
    PASS_NUM = 1
    for pass_id in range(PASS_NUM):
        total_loss_pass = 0
# data是一个list，每一个元素都是一个Tupple，房价预测的例子，前部分是影响因子，比如房子的楼层，平方等，后一个数据是价格
        for data in train_reader():
            avg_loss_value, = exe.run(
# 运行网络，正向传播，反向传播，梯度下降
                fluid.default_main_program(), 
                feed=feeder.feed(data), 
                fetch_list=[avg_cost])
            total_loss_pass += avg_loss_value
            if avg_loss_value <1.0:
                if SAVE_DIRNAME is not None:
# 存到磁盘
                    fluid.io.save_inference_model(
                        SAVE_DIRNAME, ['x'], [y_predict], exe)
                return
        print("Pass %d, total avg cost = %f" % (pass_id, total_loss_pass))
# 存到磁盘
                    fluid.io.save_inference_model(
                        SAVE_DIRNAME, ['x'], [y_predict], exe)

train_loop()

Pass 0, total avg cost = 13169.619141


In [4]:

place = fluid.CPUPlace()
exe = fluid.Executor(place)

inference_scope = fluid.core.Scope()
with fluid.scope_guard(inference_scope):
# inference_program：预测网络（正向传播的网络），feed_target_names：输入，fetch_targets：输出
    [inference_program, feed_target_names, fetch_targets] = (
# 把模型从磁盘加载回来
        fluid.io.load_inference_model(SAVE_DIRNAME, exe))
    test_reader = paddle.batch(paddle.dataset.uci_housing.test(), batch_size=20)

    test_data = test_reader().next()
    test_feat = numpy.array(map(lambda x: x[0], test_data)).astype("float32")
    test_label = numpy.array(map(lambda x: x[1], test_data)).astype("float32")
    print(feed_target_names[0])
    results = exe.run(inference_program,
                      feed={feed_target_names[0]: numpy.array(test_feat)},
                      fetch_list=fetch_targets)
    print("infer results: ", results[0])
    print("ground truth: ", test_label)

x
('infer results: ', array([[13.028505 ],
       [13.27338  ],
       [12.973402 ],
       [13.882124 ],
       [13.2918   ],
       [13.662638 ],
       [13.631207 ],
       [13.248424 ],
       [12.019205 ],
       [13.340189 ],
       [11.608441 ],
       [12.432707 ],
       [12.782322 ],
       [12.62546  ],
       [12.8106575],
       [13.021822 ],
       [13.677567 ],
       [13.598593 ],
       [14.043578 ],
       [13.059525 ]], dtype=float32))
('ground truth: ', array([[ 8.5],
       [ 5. ],
       [11.9],
       [27.9],
       [17.2],
       [27.5],
       [15. ],
       [17.2],
       [17.9],
       [16.3],
       [ 7. ],
       [ 7.2],
       [ 7.5],
       [10.4],
       [ 8.8],
       [ 8.4],
       [16.7],
       [14.2],
       [20.8],
       [13.4]], dtype=float32))


In [ ]:
def train(): 
    global UCI_TRAIN_DATA 
    load_data(paddle.dataset.common.download(URL, 'uci_housing', MD5)) 
    def reader(): 
        for d in UCI_TRAIN_DATA: 
            yield d[:-1], d[-1:] 
    return reader 
